In [1]:
client_hnum='H406239'
rel='202107'
iter='2'

In [10]:
import logging
logging.basicConfig( format='%(asctime)s %(message)s'\
                    ,datefmt='%m/%d/%Y %I:%M:%S %p'\
                    ,level=logging.INFO)

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from pivottablejs import pivot_ui
import matplotlib.pyplot as plt

In [3]:
rnb_rev=pd.read_csv( "C:/Users/camendol/Documents/Optum/Methods/Inpatient Analytics/Claims_RNB_Analysis/acute_rnb_codedesc.txt"
                    ,sep='\t'
                    ,header=None
                    ,names=['X1','X2']
                    ,dtype=str)

In [4]:
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Users\camendol\oracle\instantclient_19_11")

dsn_tns = cx_Oracle.makedsn('som-racload03.humedica.net'
                            ,'1521'
                            ,service_name='anltsts'
                            ) 

conn = cx_Oracle.connect( user=r'camendola'
                         ,password='B2sP9mzYwN3'
                         ,dsn=dsn_tns) 

In [5]:
inp_evt_enc_clm=f"""
select pce.clinical_event_id as fin_clinical_event_id
      ,enc.DATASRC as fin_DATASRC
      ,enc.CLIENT_DS_ID as fin_CLIENT_DS_ID
      ,enc.ENCOUNTERID as fin_ENCOUNTERID
      ,clm.LOCALREV as fin_LOCALREV
      ,round(pce.EVT_END_DTM - pce.EVT_ADMIT_DTM,0) 
         as STAY_DAYS
      ,EXTRACT(YEAR from pce.EVT_ADMIT_DTM) 
        as evt_year
      ,case         
         when round(pce.EVT_END_DTM - pce.EVT_ADMIT_DTM,0) < 1         
           then 'Same'       
         else 'OnePlus'       
       end DAY_TYPE   
  from
    --Clinical Events
    (select * 
       from OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L2_PAT_CLINICAL_EVENT 
       where event_type_cui = 'CH000106') pce       
      left join
    --Event-Encounter Linking  
    (select * 
       from OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_CLINICAL_EVENT_ENCOUNTER 
       where ENCOUNTERIDTYPE_CUI = 'MASTER') cee        
      on cee.ENCOUNTER_GRP_NUM = pce.clinical_event_id   
      left join
    --Encounters  
    OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_CLINICALENCOUNTER enc              
      on cee.ENCOUNTERID = enc.ENCOUNTERID            
     and cee.CLIENT_DS_ID = enc.CLIENT_DS_ID
      left join
    --Claims !granularity!  
    (select ENCOUNTERID
           ,CLIENT_DS_ID
           ,DATASRC
           ,LOCALREV
       from OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_CLAIM) clm          
      on enc.ENCOUNTERID = clm.ENCOUNTERID        
     and enc.CLIENT_DS_ID = clm.CLIENT_DS_ID  
"""

In [6]:
rev_sum=f"""
select fin_DATASRC
      ,fin_CLIENT_DS_ID
      ,fin_LOCALREV
      ,DAY_TYPE
      ,count(*) as freq 
  from ({inp_evt_enc_clm}) ie
  group by fin_DATASRC
          ,fin_CLIENT_DS_ID
          ,fin_LOCALREV 
          ,DAY_TYPE
"""

In [13]:
rev_codes=f"""
select *
  from OADW_PRD_{client_hnum}_CDR_{rel}_{iter}.L1_II_MAP_REVENUE
"""

In [11]:
logging.info("Query Start")
rev_sum = pd.read_sql(rev_sum, con=conn)
logging.info("Query End")

08/06/2021 01:46:41 PM Query Start
08/06/2021 02:04:26 PM Query End


In [14]:
rev_codes=pd.read_sql(rev_codes, con=conn)

In [15]:
conn.close()

In [16]:
rev_sum["REV_STD"]=rev_sum["FIN_LOCALREV"].str.pad(4,side='left',fillchar='0')

rev_sum_indicators=rev_sum\
                     .merge( rev_codes
                            ,'left'
                            ,left_on="REV_STD"
                            ,right_on="REVENUE")\
                     .merge( rnb_rev
                            ,'left'
                            ,left_on="REV_STD"
                            ,right_on="X1")
   
rev_sum_indicators['REV_VALID']='YES'                  
rev_sum_indicators.loc[rev_sum_indicators['REVENUE_DESC'].isnull(), 'REV_VALID'] = 'NO'  

rev_sum_indicators['REV_POP']='YES'                  
rev_sum_indicators.loc[rev_sum_indicators['REV_STD'].isnull(), 'REV_POP'] = 'NO'

rev_sum_indicators['RNB_CODE']='YES'                  
rev_sum_indicators.loc[rev_sum_indicators['X2'].isnull(), 'RNB_CODE'] = 'NO'

In [22]:
pivot_ui( rev_sum_indicators
         ,cols=['Duration_Seg']
         ,rendererName='Bar Chart'
         ,outfile_path="C:\\Users\\camendol\\Documents\\Optum\\Methods\\Inpatient Analytics\\Claims_RNB_Analysis\\H623622_REV.html")                    


In [ ]:
#Create two final summaries
## Null/Valid Report
## RNB Report      